In [1]:
#Standard Imports
import os
import sys
import pandas as pd
# from collections import defaultdict
import timeit
from docplex.cp.model import CpoModel

In [2]:
#Custom Imports
sys.path.append('../')
from src.chord import Chord
from src.musical_work_input import MusicalWorkInput
from src.cp_model import CPModel
from src.music_functions import *

In [3]:
# Importing Musical Corpus
musical_work_df = pd.read_csv("../data/sample_input.csv")
musical_corpus = []
for i, title, meter, key, tonality, first_on_beat, melody in musical_work_df.itertuples():
    musical_corpus.append(MusicalWorkInput(title, meter, key, tonality, first_on_beat, [int(x) for x in melody.split(',')]))

In [4]:
# Importing Weights
weight_df = pd.read_csv("../data/soft_constraint_weights_temp.csv")

In [5]:
# Choosing Musical Input
music = musical_corpus[0]
#music.melody = music.melody[:9]
#music.melody_len = len(music.melody)
print(music.title, music.key, music.tonality, 
      music.first_on_beat,music.melody, music.reference_note)

Ach bleib' bei unsm Herr Jesu Christ 9 major 2 [37, 37, 40, 37, 33, 35, 37, 38, 37, 35, 33, 33, 35, 37, 37, 35, 35, 33, 33, 33, 37, 35, 37, 33, 30, 32, 33, 35, 37, 33, 33, 33, 35, 37, 37, 35, 35, 33, 33, 33] 24


In [6]:
# Importing Chord Vocabulary
if music.tonality == "major":
    chord_df = pd.read_csv("../data/chord_vocabulary_major.csv", index_col = 0)
else:
    chord_df = pd.read_csv("../data/chord_vocabulary_minor.csv", index_col = 0)
chord_vocab = []
for name, note_intervals in chord_df.itertuples():
    chord_vocab.append(Chord(name, [int(x) for x in note_intervals.split(',')]))

In [7]:
# Defining dictionary of weights for each soft constraint options:
soft_constraint_w_weights = {}
for _, name, w in weight_df.itertuples(): #name population is same as soft_constraint_options
    soft_constraint_w_weights[name] = float(w)
print(soft_constraint_w_weights)
assert sum(v for v in soft_constraint_w_weights.values() if v > 0) == 100

{'chord progression': 50.0, 'chord repetition': 4.0, 'chord bass repetition': 3.0, 'leap resolution': -1.0, 'melodic movement': 8.0, 'note repetition': 3.0, 'parallel movement': -1.0, 'voice overlap': 3.0, 'adjacent bar chords': -1.0, 'chord spacing': -1.0, 'distinct notes': 10.0, 'incomplete chord': -1.0, 'voice crossing': -1.0, 'voice range': 1.0, 'second inversion': 13.0, 'first inversion': 1.0, 'chord distribution': 4.0}


In [8]:
# Defining dictionary of hard and soft constraint options:
hard_constraint_options = ['musical input', 'voice range', 'chord membership', 'first last chords',
                           'chord repetition', 'chord bass repetition', 'adjacent bar chords', 'voice crossing', 'parallel movement',
                          'chord spacing', 'incomplete chord', 'chord distribution']
hard_constraints = {x: True if x in ['musical input', 'voice range', 'chord membership', 'first last chords',
                                     'voice crossing', 'parallel movement',
                                    'chord spacing', 'incomplete chord'] else False for x in hard_constraint_options} #if x in ['musical input', 'voice range', 'chord membership', 'first last chords', 'voice crossing', 'parallel movement', 'chord spacing', 'incomplete chord'] else False 
#hard_constraints['incomplete chord'] = False
#hard_constraints['chord distribution'] = False
soft_constraint_options = ['chord progression', 'chord repetition', 'chord bass repetition', 'leap resolution',
                           'melodic movement', 'note repetition', 'parallel movement', 'voice overlap', 'adjacent bar chords',
                           'chord spacing', 'distinct notes', 'incomplete chord', 'voice crossing', 'voice range',
                           'second inversion', 'first inversion', 'chord distribution']

In [9]:
# Defining penalties for chord progression
if music.tonality == "major":
    penalties_chord_progression = pd.read_csv("../data/chord_progression_major.csv", header = 1, index_col = 0)
else:
    penalties_chord_progression = pd.read_csv("../data/chord_progression_minor.csv", header = 1, index_col = 0)

penalties_chord_progression = dict(penalties_chord_progression.stack())

In [10]:
# Model
cp_model = CPModel("test", musical_corpus[0], chord_vocab, penalties_chord_progression,
                   hard_constraints,
                   soft_constraint_w_weights)

In [11]:
solution = cp_model.solve(log_output = True, TimeLimit = 10, LogVerbosity = 'Verbose')

(988,)
-------------------------------------------------------------------------------
Model constraints: 128943, variables: integer: 2920, interval: 0, sequence: 0
Solve status: Feasible
Search status: SearchStopped, stop cause: SearchStoppedByLimit
Solve time: 15.84 sec
-------------------------------------------------------------------------------
Objective values: (988,), bounds: (18,), gaps: (0.981781,)
Chords_0=0
Chords_1=0
Chords_2=0
Chords_3=0
Chords_4=0
Chords_5=1
Chords_6=0
Chords_7=1
Chords_8=0
Chords_9=1
Chords_10=0
Chords_11=0
Chords_12=1
Chords_13=0
Chords_14=0
Chords_15=1
Chords_16=1
Chords_17=0
Chords_18=0
Chords_19=0
Chords_20=0
Chords_21=1
Chords_22=0
Chords_23=0
Chords_24=1
Chords_25=4
Chords_26=0
Chords_27=1
Chords_28=0
Chords_29=0
Chords_30=0
Chords_31=0
Chords_32=1
Chords_33=0
Chords_34=0
Chords_35=1
Chords_36=1
Chords_37=0
Chords_38=0
Chords_39=0
Notes_0=37
Notes_1=37
Notes_2=40
Notes_3=37
Notes_4=33
Notes_5=35
Notes_6=37
Notes_7=38
Notes_8=37
Notes_9=35
Notes_10

voice range_144=0
voice range_145=0
voice range_146=0
voice range_147=1
voice range_148=0
voice range_149=0
voice range_150=0
voice range_151=0
voice range_152=1
voice range_153=0
voice range_154=0
voice range_155=1
voice range_156=1
voice range_157=0
voice range_158=0
voice range_159=0
None


In [12]:
result = cp_model.get_solution()

In [13]:
print(result['Chords'])

['I', 'I', 'I', 'I', 'I', 'ii', 'I', 'ii', 'I', 'ii', 'I', 'I', 'ii', 'I', 'I', 'ii', 'ii', 'I', 'I', 'I', 'I', 'ii', 'I', 'I', 'ii', 'iii', 'I', 'ii', 'I', 'I', 'I', 'I', 'ii', 'I', 'I', 'ii', 'ii', 'I', 'I', 'I']


In [14]:
{k: sum(sum(x) for x in v) for k, v in result['Penalties'].items()}

{'chord progression': 115,
 'chord repetition': 72,
 'chord bass repetition': 21,
 'leap resolution': 0,
 'melodic movement': 301,
 'note repetition': 18,
 'parallel movement': 0,
 'voice overlap': 9,
 'adjacent bar chords': 0,
 'chord spacing': 0,
 'distinct notes': 80,
 'incomplete chord': 0,
 'voice crossing': 0,
 'voice range': 10,
 'second inversion': 143,
 'first inversion': 7,
 'chord distribution': 212}

In [15]:
chord = chord_vocab[7]
print(chord.note_intervals)
note = sorted(list(chord.note_intervals))[-1]
print(note)
note = (note + 9) % 12
print(note)

[7, 11, 2]
11
8


In [16]:
cp_model.export_midi()

In [17]:
actual_solution = ['I', 'I', 'V', 'I', 'IV', 'bVII', 'I', 'IV',
                   'I', 'V', 'vi', 'IV', 'IV', 'I', 'I', 'V',
                   'V', 'I', 'I', 'I', 'I', 'V', 'III', 'vi',
                   'II', 'V', '#iv_dim', 'V', 'III', 'vi', 'IV', 'I',
                   'I', 'vi', 'I', 'V', 'V', 'I', 'I', 'I']
count = 0
for i, predicted, actual in zip(range(40), cp_model.get_solution()['Chords'], actual_solution):
    if predicted != actual:
        print(i, predicted, actual)
    else:
        count += 1
similarity_score = count/40
print(similarity_score)

2 I V
4 I IV
5 ii bVII
7 ii IV
9 ii V
10 I vi
11 I IV
12 ii IV
15 ii V
16 ii V
21 ii V
22 I III
23 I vi
24 ii II
25 iii V
26 I #iv_dim
27 ii V
28 I III
29 I vi
30 I IV
32 ii I
33 I vi
35 ii V
36 ii V
0.4
